In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

from helpers import feature_importance, pearson_correlation, transform_2D, seed_everything, scaler
from calculate import ml_predict, val_score, calc_average
from nn import create_model, train_model_mae, train_model_rmse, load_model

# specify image save directory
#image_directory = "D:\Google Drive\Homework\Lab\Cu-Al\CO2RR-Adsorbates-main\images"
# Set fonts to Arial
matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"

# Seed all parameters to ensure repeatability
seed_everything()

# Read in our data file
df = pd.read_csv("/content/CO Dimerization Training.csv")

# Drop adsorbates 1 and 2 from our feature space as they will not participate in training
df = df.drop(columns = ['Adsorbate 1', 'Adsorbate 2'])

final_df = df

# Boxplot of the target feature
final_df.boxplot(color=dict(boxes='black', whiskers='black', medians='black', caps='black'), 
                      column='G_C2O2 - Gco', grid=False, figsize=(20, 15));
# Adding gaussian noise to target values to prevent overfitting

np.random.seed(1)
mu, sigma = 0, 0.065
noise = np.random.normal(mu, sigma, [1,65]).tolist()
final_df['G_C2O2 - Gco'] = final_df['G_C2O2 - Gco'] + noise[0]

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.kernel_ridge import KernelRidge

# Instantiate regressor algorithms
GBR = GradientBoostingRegressor(random_state=42)
KNR = KNeighborsRegressor()
RFR = RandomForestRegressor(random_state=42)
SR = SVR()
KRR = KernelRidge()
xgbr = XGBRegressor(random_state=42)
light = LGBMRegressor(random_state=42)

regressors = [GBR, KNR, RFR, SR, KRR, xgbr, light]
reg_xgbr = [xgbr]

# Create an empty df to save the results of each regressor for each round of feature removal
results_df = pd.DataFrame(columns = ['# Features', 'GBR', 'KNR', 'RFR', 'SVR', 'KRR', 'XGBR', 'LightGBM', 'Time'])

results_xgbr = pd.DataFrame(columns = ['# Features', 'XGBR', 'Time'])


val_score(final_df, 1, reg_xgbr, results_xgbr,10)

results_xgbr
# Model performance evalution using CV
val_score(final_df, 1, regressors, results_df, 10)

# Check the results of model performance
results_df

# Print a ranking of initial feature importance
feature_importance(XGBRegressor(), final_df)
# Pearson's correlation heatmap of feature pairs
pearson_correlation(final_df, last=True)

# round2
# make a copy of the df from the previoud edition
df_v2 = final_df.copy()
# Dropping low ranking or dependent features
df_v2 = df_v2.drop(columns = ['VdW Radius 1 add VdW Radius 2',
       'FIE 1 add FIE 2', 'EA 1 add EA 2',
       'num of Valence 1 add num of Valence 2',
       'IE 1 add IE 2', 'Average EA'])
val_score(df_v2, 2, reg_xgbr, results_xgbr, 10)
results_xgbr
val_score(df_v2, 2, regressors, results_df, 10)
results_df
feature_importance(XGBRegressor(), df_v2)
pearson_correlation(df_v2, last=True)

# round3
df_v3 = df_v2.copy()
df_v3 = df_v3.drop(columns = ['PE 1 add PE 2',
       'Covalent Radius 1 add Covalent Radius 2', 'Average EA of Adsorbates squared', 'Average PE of Adsorbates squared',
       'Average Covalent Radius of Adosrbates squared',
       'Average FIE of Adsorbates squared',
       'Average Atomic num of Adsorbates squared',
       'Average num of Valence Electron squared', 'Average IE squared', 'num of Valence Electron 1', 'num of Valence Electron 2'])
val_score(df_v3, 3, reg_xgbr, results_xgbr, 10)
results_xgbr
val_score(df_v3, 3, regressors, results_df, 10)
results_df
feature_importance(XGBRegressor(), df_v3)

# round4
df_v4 = df_v3.copy()
df_v4 = df_v4.drop(columns = ['num of H 1', 'num of H 2', 'num of H', 'VdW Radius 1',
       'VdW Radius 2', 'IE 1', 'IE 2', 'PE 1', 'PE 2'])
val_score(df_v4, 4, reg_xgbr, results_xgbr, 10)
results_xgbr
val_score(df_v4, 4, regressors, results_df, 10)
results_df
feature_importance(XGBRegressor(), df_v4)

# round5
df_v5 = df_v4.copy()
df_v5 = df_v5.drop(columns = ['Average PE', 'FIE 1', 'FIE 2', 'num of NM atoms 1', 'num of NM atoms 2', 'Average Atomic num'])
val_score(df_v5, 5, reg_xgbr, results_xgbr, 10)
results_xgbr
val_score(df_v5, 5, regressors, results_df, 10)
results_df
feature_importance(XGBRegressor(), df_v5)

# round6
df_v6 = df_v5.copy()
df_v6 = df_v6.drop(columns = ['Atomic num 1', 'VdW Radius 1 minus VdW Radius 2', 'Average FIE', 'Atomic num 2', 'FIE 1 minus FIE 2', 
                              'Covalent Radius 1', 'Covalent Radius 2', 
                              'EA 1', 'Average VdW Radius squared', 'EA 2', 'num of Atoms'])
val_score(df_v6, 6, reg_xgbr, results_xgbr, 10)
results_xgbr
val_score(df_v6, 6, regressors, results_df, 10)
results_df
feature_importance(XGBRegressor(), df_v6)
pearson_correlation(df_v6, last=True)

# round7
df_v7 = df_v6.copy()
df_v7 = df_v7.drop(columns = ['Average VdW Radius'])
val_score(df_v7, 7, reg_xgbr, results_xgbr, 10)
results_xgbr
val_score(df_v7, 7, regressors, results_df, 10)
results_df
feature_importance(XGBRegressor(), df_v7)
pearson_correlation(df_v7, last=True)

# round8
df_v8 = df_v7.copy()
df_v8 = df_v8.drop(columns = [ 'PE 1 minus PE 2'])
val_score(df_v8, 8, reg_xgbr, results_xgbr, 10)
results_xgbr
val_score(df_v8, 8, regressors, results_df, 10)
results_df
feature_importance(XGBRegressor(), df_v8)
pearson_correlation(df_v8, last=True)

# round9
df_v9 = df_v8.copy()
df_v9 = df_v9.drop(columns = ['IE 1 minus IE 2'])
val_score(df_v9, 9, reg_xgbr, results_xgbr, 10)
results_xgbr
val_score(df_v9, 9, regressors, results_df, 10)
results_df
feature_importance(XGBRegressor(), df_v9)
pearson_correlation(df_v9, last=True)

# round10
df_v10 = df_v9.copy()
df_v10 = df_v10.drop(columns = ['Average IE'])
val_score(df_v10, 10, reg_xgbr, results_xgbr, 10)
results_xgbr
val_score(df_v10, 10, regressors, results_df, 10)
results_df
feature_importance(XGBRegressor(), df_v10)
pearson_correlation(df_v10, last=True)

#df.columns = df.columns.str.strip()

# Get XGBR MAE values
mae_0 = results_df['XGBR'][1][0]
mae_1 = results_df['XGBR'][2][0]
mae_2 = results_df['XGBR'][3][0]
mae_3 = results_df['XGBR'][4][0]
mae_4 = results_df['XGBR'][5][0]
mae_5 = results_df['XGBR'][6][0]
mae_6 = results_df['XGBR'][7][0]
mae_7 = results_df['XGBR'][8][0]
mae_8 = results_df['XGBR'][9][0]
mae_9 = results_df['XGBR'][10][0]
mae = [mae_0, mae_1, mae_2, mae_3, mae_4, mae_5, mae_6, mae_7, mae_8, mae_9]

# Get XGBR RMSE values
rmse_0 = results_df['XGBR'][1][1]
rmse_1 = results_df['XGBR'][2][1]
rmse_2 = results_df['XGBR'][3][1]
rmse_3 = results_df['XGBR'][4][1]
rmse_4 = results_df['XGBR'][5][1]
rmse_5 = results_df['XGBR'][6][1]
rmse_6 = results_df['XGBR'][7][1]
rmse_7 = results_df['XGBR'][8][1]
rmse_8 = results_df['XGBR'][9][1]
rmse_9 = results_df['XGBR'][10][1]
rmse = [rmse_0, rmse_1, rmse_2, rmse_3, rmse_4, rmse_5, rmse_6, rmse_7, rmse_8, rmse_9]

# Get run times
time1 = results_xgbr['Time'][1]
time2 = results_xgbr['Time'][2]
time3 = results_xgbr['Time'][3]
time4 = results_xgbr['Time'][4]
time5 = results_xgbr['Time'][5]
time6 = results_xgbr['Time'][6]
time7 = results_xgbr['Time'][7]
time8 = results_xgbr['Time'][8]
time9 = results_xgbr['Time'][9]
time10 = results_xgbr['Time'][10]
run_time = [time1, time2, time3, time4, time5, time6, time7, time8, time9, time10]

# Get # of features
feature1 = str(results_df['# Features'][1])
feature2 = str(results_df['# Features'][2])
feature3 = str(results_df['# Features'][3])
feature4 = str(results_df['# Features'][4])
feature5 = str(results_df['# Features'][5])
feature6 = str(results_df['# Features'][6])
feature7 = str(results_df['# Features'][7])
feature8 = str(results_df['# Features'][8])
feature9 = str(results_df['# Features'][9])
feature10 = str(results_df['# Features'][10])
ml_features = [feature1, feature2, feature3, feature4, feature5, feature6, feature7, feature8, feature9, feature10]

# Plot figure
ml_features = np.array(ml_features)
run_time = np.array(run_time)
data1 = np.array(rmse)
data2 = np.array(mae)

m1_t = pd.DataFrame({
 'run_time' : run_time,
 'RMSE (eV)' : data1,
 'MAE (eV)' : data2})

fig, ax1 = plt.subplots()

bar1 = ax1.plot(ml_features, m1_t['RMSE (eV)'], color='#5D98A6', label='RMSE', 
                marker = 'o', linewidth = 8, markersize = 35)
bar2 = ax1.plot(ml_features, m1_t['MAE (eV)'], color='#F2C36B', label='MAE', 
                marker = 'o', linewidth = 8, markersize = 35)

ax1.set_xticklabels(ml_features)
ax1.tick_params(axis='y', labelcolor='black', labelsize = 45, width=6, length=15, color='black', which='major')
ax1.tick_params(axis='x', labelcolor='black', labelsize = 45, width=6, length=15, color='black', which='major')
ax1.tick_params(axis='y', labelcolor='black', labelsize = 45, width=6, length=8, color='black', which="minor")

ax1.minorticks_on()
ax1.set_yticks([0.05, 0.06, 0.07, 0.08, 0.09, 0.1])

ax2 = ax1.twinx()

linecolor = '#CD5C5C'
time = ax2.plot(ml_features, m1_t['run_time'], marker = 'o', color = linecolor, label='Time', linewidth = 8, markersize = 35)
ax2.tick_params(axis='y', labelcolor=linecolor, labelsize = 45, width=6, length=15, color=linecolor, which='major')
ax2.tick_params(axis='y', labelcolor=linecolor, labelsize = 45, width=6, length=8, color=linecolor, which="minor")

lns = bar1+bar2+time
labs = [l.get_label() for l in lns]
ax2.legend(lns, labs, loc=0, fontsize=40, frameon=False)


ax1.set_xlabel('Number of Features', size = 55)
ax1.set_ylabel('Error (eV)', color='black', size = 55)
ax2.set_ylabel('Time (s)', color=linecolor, size = 55)
ax2.minorticks_on()
ax2.set_yticks([2, 3, 4])

for axis in ['top','bottom','left','right']:
    if axis == 'right':
        ax1.spines[axis].set_color(linecolor)
    ax1.spines[axis].set_linewidth(6)

plt.savefig('{}dim_feature_selection.png'.format(image_directory),dpi=600, bbox_inches = "tight")
plt.show()

# Save errors to csv
results_df.to_csv('CO Dimerization Errors.csv')

# Plot final feature importance figure

importance_labels = ['$N_{VE_{avg}}$', '$N_{VE_{1}}$ $-$ $N_{VE_{2}}$', '$r_1$ $-$ $r_2$', '$IE_{avg}$', 
                     '$r_{avg}}$',  '$EA_{1}$ $-$ $EA_{2}$']
importance_labels = reversed(importance_labels)

features = df_v9.iloc[:, 0:-1]
target = df_v9.iloc[:, -1]

xgbr.fit(features, target)

importance = xgbr.feature_importances_
features = df_v9.columns[0:-1]

col = zip(features, importance)

# sort and save features based on its importance into a dataframe
importance_data = pd.DataFrame(col, columns=['feature', 'importance'])
importance_data_asc = importance_data.sort_values('importance')

# plot the importance data
fig, ax = plt.subplots()
plt.barh(y=importance_data_asc['feature'], width=importance_data_asc['importance'], height=0.9, color='#377488')
ax.set_yticklabels(importance_labels)

for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(6)

ax.tick_params(axis='x', colors='black', width=6, length=15, color='black')
ax.tick_params(axis='y', colors='black', width=6, length=15, color='black')    

ax.set_xlabel("Feature Importance", fontsize = 75)
ax.set_ylabel("Features", fontsize = 75)

ax.tick_params(axis='x', colors='black', labelsize = 65, width=6, length=15, color='black')
ax.tick_params(axis='y', colors='black', labelsize = 65, width=6, length=15, color='black')

plt.savefig('{}dim_final_features.png'.format(image_directory),dpi=600, bbox_inches = "tight")
plt.show();

# Plot final feature importance figure

importance_labels = ['$N_{VE_{avg}}$', '$N_{VE_{1}}$ $-$ $N_{VE_{2}}$', '$r_1$ $-$ $r_2$', '$IE_{avg}$', 
                     '$r_{avg}}$',  '$EA_{1}$ $-$ $EA_{2}$']
importance_labels = reversed(importance_labels)

features = df_v9.iloc[:, 0:-1]
target = df_v9.iloc[:, -1]

xgbr.fit(features, target)

importance = xgbr.feature_importances_
features = df_v9.columns[0:-1]

col = zip(features, importance)

# sort and save features based on its importance into a dataframe
importance_data = pd.DataFrame(col, columns=['feature', 'importance'])
importance_data_asc = importance_data.sort_values('importance')

# plot the importance data
fig, ax = plt.subplots()
plt.barh(y=importance_data_asc['feature'], width=importance_data_asc['importance'], height=0.9, color='#377488')
ax.set_yticklabels(importance_labels)

for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(6)

ax.tick_params(axis='x', colors='black', width=6, length=15, color='black')
ax.tick_params(axis='y', colors='black', width=6, length=15, color='black')    

ax.set_xlabel("Feature Importance", fontsize = 75)
ax.set_ylabel("Features", fontsize = 75)

ax.tick_params(axis='x', colors='black', labelsize = 65, width=6, length=15, color='black')
ax.tick_params(axis='y', colors='black', labelsize = 65, width=6, length=15, color='black')

plt.savefig('{}dim_final_features.png'.format(image_directory),dpi=600, bbox_inches = "tight")
plt.show();

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
import xgboost
import lightgbm

features = df_v9.iloc[:, 0:-1]
target = df_v9.iloc[:, -1]

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import RepeatedKFold


# Defining XGBR hyperparameters
xgbr_grid = {
    
    'min_child_weight': [1, 5, 10],
    'n_estimators': [100, 200, 300],
    'max_depth': [4, 6, 8, 10],
    'learning_rate': [0.3, 1, 2],
    'colsample_bytree': [0.1, 0.5, 0.8, 1],
    'colsample_bylevel': [0.1, 0.5, 0.8, 1],

}

# Defining fold parameters
fold = RepeatedKFold(n_splits = 10, n_repeats = 1, random_state = 42)


# Performing Randomized CV search on XGBR
xgbr_grid_cv = GridSearchCV(estimator = xgbr, cv = fold,
            param_grid = xgbr_grid, n_jobs = -1, scoring='neg_root_mean_squared_error',
            verbose = 5, return_train_score = True)

xgbr_grid_cv.fit(features, target)

from sklearn.model_selection import cross_val_score

xgbr_optimized = xgbr_grid_cv.best_estimator_

rmse_score = cross_val_score(xgbr_optimized, features, target, cv = fold, scoring='neg_root_mean_squared_error')
mae_score = cross_val_score(xgbr_optimized, features, target, cv = fold, scoring='neg_mean_absolute_error')

optimized_xgbr_rmse = np.abs(rmse_score).mean()
optimized_xgbr_mae = np.abs(mae_score).mean()



from sklearn.model_selection import train_test_split

r2 = []
rmse = []
mae = []


for i in [0.5, 0.667, 0.75, 0.8, 0.83, 0.857, 0.875, 0.889, 0.9]:
    r2_temp = []
    rmse_temp = []
    mae_temp = []
    for j in range(50):
        X_train, X_test, y_train, y_test = train_test_split(features, target, train_size = i, random_state = j)
        xgbr.fit(X_train, y_train)

        # Predicting training data
        train_data = X_train.copy()
        train_data['G_co (eV)'] = y_train.copy()
        train_data['prediction'] = xgbr.predict(train_data.iloc[:, :-1])    

        # Predicting testing data
        test_data = X_test.copy()
        test_data['G_co (eV)'] = y_test.copy()
        test_data['prediction'] = xgbr.predict(test_data.iloc[:, :-1])
        test_data['mae_error'] = np.abs(test_data['G_co (eV)'] - test_data['prediction'])
        test_data['rmse_error'] = (test_data['G_co (eV)'] - test_data['prediction']) ** 2

        mae_temp.append(test_data['mae_error'].mean())
        rmse_temp.append(np.sqrt(test_data['rmse_error'].mean()))

        x = pd.concat([train_data['G_co (eV)'], test_data['G_co (eV)']])
        y = pd.concat([train_data['prediction'], test_data['prediction']])

        # Calculate R Squared
        coeffs = np.polyfit(x, y, 1)
        p = np.poly1d(coeffs)

        ybar = np.sum(y) / len(y)
        ssreg = np.sum((p(x) - ybar) ** 2)
        sstot = np.sum((y - ybar) ** 2)
        Rsqr = ssreg / sstot

        r2_temp.append(Rsqr)
    
    r2.append(np.array(r2_temp).mean())
    rmse.append(np.array(rmse_temp).mean())
    mae.append(np.array(mae_temp).mean())

from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

t = [2, 3, 4, 5, 6, 7, 8, 9, 10]
data1 = mae
data2 = r2
data3 = rmse

fig, ax1 = plt.subplots()

ax1.set_xlabel('Number of Splits', size = 55)
ax1.set_ylabel('Error (eV)', color='black', size = 55)
line1 = ax1.plot(t, data1, color='#5D98A6', marker = 'o', linewidth = 8, markersize = 35, label = 'MAE')
line2 = ax1.plot(t, data3, color='#F2C36B', marker = 'o', linewidth = 8, markersize = 35, label = 'RMSE')
ax1.tick_params(axis='y', labelcolor='black', labelsize = 45, width=6, length=15, color='black', which="major")
ax1.tick_params(axis='y', labelcolor='black', labelsize = 45, width=6, length=8, color='black', which="minor")

ax1.tick_params(axis='x', colors='black', labelsize = 45, width=6, length=15, color='black')
for axis in ['top','bottom','left','right']:
    ax1.spines[axis].set_linewidth(6)


ax1.minorticks_on()
ax1.set_yticks([0, 0.05, 0.1, 0.15, 0.2])
ax1.set_xticks([2, 3, 4, 5, 6, 7, 8, 9, 10])
ax1.set_xticklabels([2, 3, 4, 5, 6, 7, 8, 9, 10])

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('$R^2$', color=color, size = 55)  # we already handled the x-label with ax1
line3 = ax2.plot(t, data2, color=color, marker = 'o', linewidth = 8, markersize = 35, label = '$R^2$')
ax2.tick_params(axis='y', labelcolor=color, labelsize = 45, width=6, length=15, color=color, which='major')
ax2.tick_params(axis='y', labelcolor=color, labelsize = 45, width=6, length=8, color=color, which="minor")

lns = line1+line2+line3
labs = [l.get_label() for l in lns]
ax2.legend(lns, labs, loc=0, fontsize=40, frameon=False)

for axis in ['top','bottom','left','right']:
    if axis == 'right':
        ax1.spines[axis].set_color(color)
    ax1.spines[axis].set_linewidth(6)

ax2.minorticks_on()
ax2.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
plt.xlim(1, 11)
plt.ylim(0, 1);

plt.savefig('{}dim_train_split.png'.format(image_directory),dpi=600, bbox_inches = "tight")
plt.show()



from sklearn.model_selection import train_test_split

# Split training data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, target, train_size = 0.9, random_state = 30)


import time

# Training neural network to minimize RMSE
start = time.time()
model_history_rmse = train_model_rmse(X_train, y_train, num_cols = len(features.columns))
end = time.time()

print('Time taken to train this neural network was {} seconds'.format(int(end - start)))


# Find the minimum RMSE
nn_RMSE = min(model_history_rmse.history['val_loss'])

# Training neural network to minimize MAE
start = time.time()
model_history_mae = train_model_mae(X_train, y_train, num_cols = len(features.columns))
end = time.time()

print('Time taken to train this neural network was {} seconds'.format(int(end - start)))

# Find the minimum MAE
nn_MAE = min(model_history_mae.history['val_loss'])


training_losses = model_history_mae.history['loss']
validation_losses = model_history_mae.history['val_loss']

fig, ax = plt.subplots()
plt.plot(training_losses, label="Training Loss", linewidth = 2.5)
plt.plot(validation_losses, label="Validation Loss", linewidth = 2.5, c='g')
#plt.title('Learning Curve', fontsize = 20)
ax.set_xlabel('Epoch', fontsize = 45)
ax.set_ylabel('MAE (eV)', fontsize = 45)

ax.tick_params(axis='x', colors='black', labelsize = 35, width=6, length=15, color='black')
ax.tick_params(axis='y', colors='black', labelsize = 35, width=6, length=15, color='black')

for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(6)

plt.savefig('{}dim_nn_mae.png'.format(image_directory),dpi=600, bbox_inches = "tight")
plt.legend(frameon=False, fontsize = 40);

training_losses = model_history_rmse.history['loss']
validation_losses = model_history_rmse.history['val_loss']

fig, ax = plt.subplots()
plt.plot(training_losses, label="Training Loss", linewidth = 2.5)
plt.plot(validation_losses, label="Validation Loss", linewidth = 2.5, c='g')
#plt.title('Learning Curve', fontsize = 20)
ax.set_xlabel('Epoch', fontsize = 45)
ax.set_ylabel('RMSE (eV)', fontsize = 45)

ax.tick_params(axis='x', colors='black', labelsize = 35, width=6, length=15, color='black')
ax.tick_params(axis='y', colors='black', labelsize = 35, width=6, length=15, color='black')

for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(6)

plt.savefig('{}dim_nn_rmse.png'.format(image_directory),dpi=600, bbox_inches = "tight")
plt.legend(frameon=False, fontsize = 40);

# Train on models
GBR.fit(X_train, y_train)
KNR.fit(X_train, y_train)
RFR.fit(X_train, y_train)
SR.fit(X_train, y_train)
KRR.fit(X_train, y_train)
xgbr.fit(X_train, y_train)
#xgbr_optimized.fit(X_train, y_train)
light.fit(X_train, y_train)
nn = load_model('model_best_weights.h5', num_cols = len(features.columns))

def trendline_dimerization(xd, yd, i, name, rmse, ax, order=1, c='k', alpha=1, Rval=False):
    """Make a line of best fit"""

    # Calculate trendline
    coeffs = np.polyfit(xd, yd, order)

    intercept = coeffs[-1]
    slope = coeffs[-2]
    power = coeffs[0] if order == 2 else 0

    minxd = np.min(xd)
    maxxd = np.max(xd)

    xl = np.array([minxd, maxxd])
    yl = power * xl ** 2 + slope * xl + intercept

    # Plot trendline
    if i <= 3:
        ax[0, i].plot(xl, yl, c, alpha=alpha, linestyle='--', linewidth=0.8)

    if i > 3:
        ax[1, i - 4].plot(xl, yl, c, alpha=alpha, linestyle='--', linewidth=0.8)

    # Calculate R Squared
    p = np.poly1d(coeffs)

    ybar = np.sum(yd) / len(yd)
    ssreg = np.sum((p(xd) - ybar) ** 2)
    sstot = np.sum((yd - ybar) ** 2)
    Rsqr = ssreg / sstot

    if not Rval:
        # Plot R^2 value
        if i <= 3:
            ax[0, i].text(0.12, 0.01, name, fontsize=18)
            ax[0, i].text(0.12, -0.06, '$MAE = %0.3f$ eV' % mae, fontsize=18)
            ax[0, i].text(0.12, -0.025, '$RMSE = %0.3f$ eV' % rmse, fontsize=18)
            ax[0, i].text(0.12, -0.095, '$R^2 = %0.3f$' % Rsqr, fontsize=18)
        if i > 3:
            ax[1, i - 4].text(0.12, 0.01, name, fontsize=18)
            ax[1, i - 4].text(0.12, -0.06, '$MAE = %0.3f$ eV' % mae, fontsize=18)
            ax[1, i - 4].text(0.12, -0.025, '$RMSE = %0.3f$ eV' % rmse, fontsize=18)
            ax[1, i - 4].text(0.12, -0.095, '$R^2 = %0.3f$' % Rsqr, fontsize=18)
    else:
        # Return the R^2 value:
        return Rsqr


def plot_data_dimerization(train_data, test_data, i, name, rmse, mae, ax):
    if i <= 3:
        # Plot data
        ax[0, i].scatter(train_data['G_co (eV)'], train_data['prediction'], label="Training Set",
                         alpha=1, marker='o', s=35, facecolor='none', color='#1f77b4')

        ax[0, i].scatter(test_data['G_co (eV)'], test_data['prediction'], label='Validation Set',
                         alpha=1, marker='s', s=35, facecolor='none', color='#ff7f0e')
        ax[0, i].tick_params(axis='y', labelcolor='black', labelsize=25, width=2, length=6, color='black',
                             which='major')
        ax[0, i].tick_params(axis='x', labelcolor='black', labelsize=25, width=2, length=6, color='black',
                             which='major')

        for axis in ['top', 'bottom', 'left', 'right']:
            ax[0, i].spines[axis].set_linewidth(2)

        ax[0, i].set_yticks([-0.1, 0, 0.1, 0.2, 0.3, 0.4])
        ax[0, i].set_xticks([-0.1, 0, 0.1, 0.2, 0.3, 0.4])

        # Combine data for best fit line
        x = pd.concat([train_data['G_co (eV)'], test_data['G_co (eV)']])
        y = pd.concat([train_data['prediction'], test_data['prediction']])

        # plot best fit line
        trendline_dimerization(x, y, i, name, rmse, ax)

        # plot legend/axis
        ax[0, i].legend(loc='upper left', fontsize=18, frameon=False)

    if i > 3:
        # Plot data
        ax[1, i - 4].scatter(train_data['G_co (eV)'], train_data['prediction'], label="Training Set",
                             alpha=1, marker='o', s=35, facecolor='none', color='#1f77b4')
        ax[1, i - 4].scatter(test_data['G_co (eV)'], test_data['prediction'], label='Validation Set',
                             alpha=1, marker='s', s=35, facecolor='none', color='#ff7f0e')
        ax[1, i - 4].tick_params(axis='y', labelcolor='black', labelsize=25, width=2, length=6, color='black',
                                 which='major')
        ax[1, i - 4].tick_params(axis='x', labelcolor='black', labelsize=25, width=2, length=6, color='black',
                                 which='major')

        for axis in ['top', 'bottom', 'left', 'right']:
            ax[1, i - 4].spines[axis].set_linewidth(2)

        ax[1, i - 4].set_yticks([-0.1, 0, 0.1, 0.2, 0.3, 0.4])
        ax[1, i - 4].set_xticks([-0.1, 0, 0.1, 0.2, 0.3, 0.4])

        # Combine data to create best fit line
        x = pd.concat([train_data['G_co (eV)'], test_data['G_co (eV)']])
        y = pd.concat([train_data['prediction'], test_data['prediction']])

        # Plot best fit line
        trendline_dimerization(x, y, i, name, rmse, ax)

        # Plot legend/axis
        ax[1, i - 4].legend(loc='upper left', fontsize=18, frameon=False)


fig, ax = plt.subplots(nrows=2, ncols=4, sharex = True, sharey = True, figsize = (20, 10))

regressors = [xgbr, GBR, RFR, KRR, KNR, SR, light, nn]

names = {
    
    GBR: 'GBR',
    KNR: 'KNR',
    RFR: 'RFR',
    SR: 'SVR',
    light: 'LightGBM',
    xgbr: 'XGBR',
    KRR: 'KRR',
    nn: 'NN'
    
}

RMSE = {
    
    GBR: results_df['GBR'][9][1],
    KNR: results_df['KNR'][9][1],
    RFR: results_df['RFR'][9][1],
    SR: results_df['SVR'][9][1],
    light: results_df['LightGBM'][9][1],
    xgbr: results_df['XGBR'][9][1],
    KRR: results_df['KRR'][9][1],
    nn: nn_RMSE
    
}

MAE = {
    
    GBR: results_df['GBR'][9][0],
    KNR: results_df['KNR'][9][0],
    RFR: results_df['RFR'][9][0],
    SR: results_df['SVR'][9][0],
    light: results_df['LightGBM'][9][0],
    xgbr: results_df['XGBR'][9][0],
    KRR: results_df['KRR'][9][0],
    nn: nn_MAE
    
}

# Make Gco predictions on training and testing data using each regressor
for i, reg in enumerate(regressors):
    
    name = names[reg]
    rmse = RMSE[reg]
    mae = MAE[reg]
    
    
    # Predicting training data
    train_data = X_train.copy()
    train_data['G_co (eV)'] = y_train.copy()
    train_data['prediction'] = reg.predict(train_data.iloc[:, :-1])    
    
    # Predicting testing data
    test_data = X_test.copy()
    test_data['G_co (eV)'] = y_test.copy()
    test_data['prediction'] = reg.predict(test_data.iloc[:, :-1])    
    
    # Plotting the performance of each regressor
    plot_data_dimerization(train_data, test_data, i, name, rmse, mae, ax)

fig.text(0.5, -0.03, r'$\Delta G_{\mathrm{CO\;Dimerization}}^{\mathrm{DFT}}$ (eV)', ha='center', va='center', fontsize = 35)
fig.text(-0.02, 0.5, r'$\Delta G_{\mathrm{CO\;Dimerization}}^{\mathrm{ML}}$ (eV)', ha='center', va='center', rotation='vertical', fontsize = 35)
fig.tight_layout(pad = 1)

plt.savefig('{}dim_model_performances.png'.format(image_directory),dpi=600, bbox_inches = "tight")


rmse_grid = -cross_val_score(xgbr, features, target, cv=RepeatedKFold(n_splits = 10, n_repeats = 5, random_state = 42), scoring='neg_mean_absolute_error')

rmse_grid = rmse_grid.tolist()
rmse_grid = np.array(['%.3f' % elem for elem in rmse_grid]).astype('float')
rmse_grid = transform_2D(rmse_grid, 11)

# Plotting all CV errors
ax = sns.heatmap(rmse_grid, linewidth=2,
                 cmap = sns.diverging_palette(20, 220, l=35, n=220), square = True, vmin = 0, vmax = 0.1,
                 cbar=False, annot=True, fmt='.3f', annot_kws={"fontsize":20})
plt.xticks(ha='right')

x_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
y_labels = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']


ax.set_xticklabels(x_labels, rotation = 0, size = 30)
ax.set_yticklabels(y_labels, rotation = 0, size = 30)

ax.tick_params(axis='x', labelcolor='black', labelsize = 30, width=3, length=6, color='black') 
ax.tick_params(axis='y', labelcolor='black', labelsize = 30, width=3, length=6, color='black') 


ax.set_xlabel('Split #', size = 35)
ax.set_ylabel('Repeat #', color='black', size = 35)

plt.savefig('{}dim_cv_splits.png'.format(image_directory),dpi=600, bbox_inches = "tight")
plt.show()

# Import full data
prediction_df = pd.read_csv("/content/CO Dimerization Full Data.csv")
adsorbates = prediction_df[['Adsorbate 1', 'Adsorbate 2']]
# Savings the Gco column
targets = prediction_df['G_C2O2 - Gco']
# Removing all columns that were deleted in feature selection to maintain the same prediction features as training
prediction_df = prediction_df.drop(prediction_df.columns.difference(test_data.columns), 1)
# Plot of the statistical distibution between training dataset and full dataset
fig, ax = plt.subplots(nrows=1, ncols=2, sharey = True, sharex = True, figsize = (20, 15))

# minmax scaling training data
training_df = scaler(df_v9.copy(), 'CO Dimerization Full Data.csv', 'G_C2O2 - Gco')
training_df = training_df.iloc[:, :-1].melt(var_name='Features', value_name='Values')

# minmax scaling full data
full_df = scaler(prediction_df.copy(), 'CO Dimerization Full Data.csv', 'G_C2O2 - Gco')
full_df = full_df.melt(var_name='Features', value_name='Values')

sns.violinplot(x = 'Values', y = 'Features', data = training_df, ax=ax[0], color="#377488").set(xticklabels=[], title='Training and Validation Data (64 data pts)')
sns.violinplot(x = 'Values', y = 'Features', data = full_df, ax=ax[1], color="#377488").set(xticklabels=[], title='Full Prediction Data (289 data pts)')

ax[0].set_ylabel('')    
ax[0].set_xlabel('')
ax[1].set_ylabel('')    
ax[1].set_xlabel('')


ax[0].tick_params(axis='y', labelcolor='black', labelsize = 45, which='both', width=6, length=8)
ax[0].tick_params(axis='x', width=0, length=0)
ax[1].tick_params(width=0, length=0)
for axis in ['top','bottom','left','right']:
    ax[0].spines[axis].set_linewidth(6)
    ax[1].spines[axis].set_linewidth(6)

ax[0].set_yticklabels(corr_labels)
    
ax[0].set_title('Training and Validation Data (64 data pts)', fontsize=30)    
ax[1].set_title('Full Prediction Data (289 data pts)', fontsize=30)

plt.xticks(fontsize=40)

fig.text(0.65, -0.03, 'Normalized Relative Values', ha='center', va='center', fontsize = 55)
fig.text(-0.03, 0.5, 'Features', ha='center', va='center', rotation='vertical', fontsize = 55)
fig.tight_layout(pad = 1)

plt.savefig('{}dim_violin.png'.format(image_directory),dpi=600, bbox_inches = "tight")

# Adding the Gco column back
prediction_df['G_C2O2 - Gco'] = targets

# Full dataset prediction
import time

start = time.time()
ml_predict(prediction_df, 'G_C2O2 - Gco' ,xgbr, 'Gc-c_ML', 'Gc-c_Final')
end = time.time()
total = end-start

# obtain the relative RMSE of the full dataset based on training set (not strictly accurate)
b = np.sqrt(((prediction_df['G_C2O2 - Gco'] - prediction_df['Gc-c_ML'])**2).mean()).round(4)
print('Training RMSE: {}'.format(results_df['XGBR'][9][1].round(4)))
print('Testing RMSE: {}'.format(b))

prediction_df.boxplot(color=dict(boxes='black', whiskers='black', medians='black', caps='black'), 
                      column='Gc-c_Final', grid=False, figsize=(20, 15));

# Add adsorbate names back to df
final_df = adsorbates.join(prediction_df, how = 'left')

# Double check no errors in df
final_df

# Save the predicted dataframe
final_df.to_csv('predicted_Gc-c_xgbr.csv')

# Due to the variability of ML predictions, we repeated the prediction 5 times and took the average Gc-c as result
five_times = pd.read_csv('/content/dim_five_times.csv')

five_times

# Define the adsorbate names and all dimerization energies

adsorbates = ['O', 'OH', 'N', 'NH', 'NH$_2$', 'NH$_{3}$', 'C', 'CH', 'CH$_2$', 'CH$_3$', 'S',
 'SH', 'SH$_2$', 'F', 'Cl', 'Br', 'I']
dim_energy = five_times['Averaged Gco']
dim_energy_min_cu = five_times['Averaged Gco']

dim_energy.describe()
# Transform series into a 2D array
twoD = transform_2D(dim_energy_min_cu, 18)
twoD = np.array(twoD)
# count the number of combinations that exhibit catalytic performance greater than pristine Cu
final = (twoD + twoD.T) / 2 - 0.184
mask = np.tril(np.ones_like(final, dtype=bool))

count = 0
for i in range(0, 17):
    for j in range(0, 17):
        if (final[i, j] < 0) and (mask[i,j]):
            count +=1
            
count
final_dim = final + 0.184

result = []
for i in range(len(final_dim)):
    for j in range(len(final_dim[i])):
        result.append(final_dim[i, j])


result = np.array(result)
final_df['Gc-c_Final'] = result
final_df

final_df.to_csv('predicted_Gc-c_xgbr_avg.csv')

# Plot final result heatmap
import matplotlib

cmap1 = matplotlib.colors.ListedColormap(['g'])
mask = np.tril(np.ones_like(final, dtype=bool))

ax = sns.heatmap(final, linewidth=2, mask = 1 - mask,
                 cmap = "coolwarm", square = True, cbar = False, vmin=-0.4, vmax=0.4)
ax.figure.axes[-1].yaxis.label.set_size(30)

#ax = sns.heatmap(twoD, linewidth=2, mask = np.array(twoD) < 0.3, cmap = cmap1, square = True, cbar=False)

ax.set_xticklabels(adsorbates, rotation = 45, size = 30)
ax.set_yticklabels(adsorbates, rotation = 0, size = 30)

ax.tick_params(axis='x', colors='black', width=0, length=0, color='black')
ax.tick_params(axis='y', colors='black', width=0, length=0, color='black') 

divider = make_axes_locatable(ax) 
cax = divider.append_axes("right", size="5%", pad=0.1) 
cbar = plt.colorbar(ax.collections[0], cax=cax)
cbar.set_ticks([-0.4, -0.2, 0.0, 0.2, 0.4])
cbar.ax.set_yticklabels(['-0.4', '-0.2',  '0.0', '0.2', '0.4'], size=35)
cbar.ax.set_ylabel('$\Delta G_{\mathrm{CO \; Dimerization}}$ $-$ $\Delta G_{\mathrm{CO \; Dimerization}}^{\mathrm{Cu(100)}}$ (eV)', fontsize = 45)
cbar.ax.tick_params(axis='y', which='major', length=8, width = 2, pad=20)
cbar.outline.set_edgecolor('black')
cbar.outline.set_linewidth(2)

plt.savefig('{}dim_heatmap.png'.format(image_directory),dpi=600, bbox_inches = "tight")
plt.show()

#Calculate the average CO dimerization of each adsorbate

dim_avg = calc_average(dim_energy, 18)

col = zip(adsorbates, dim_avg)

# sort and save features based on its importance into a dataframe
difference = pd.DataFrame(col, columns=['adsorbates', 'difference'])
difference_data_asc = difference.sort_values('difference', ascending=False)

# plot the importance data
fig, ax = plt.subplots()
plt.barh(y=difference_data_asc['adsorbates'], width=difference_data_asc['difference'], height=0.9, color='#377488')
ax.set_yticklabels(difference_data_asc['adsorbates'])

ax.set_xticks([0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3])

for axis in ['top','bottom','left','right']:
    ax.spines[axis].set_linewidth(6)

ax.tick_params(axis='x', colors='black', width=6, length=15, color='black')
ax.tick_params(axis='y', colors='black', width=6, length=15, color='black')    

ax.set_xlabel("$\Delta G_{\mathrm{CO \; Dimerization}}$ (eV)", fontsize = 55)
ax.set_ylabel('Adsorbates', fontsize = 55)

ax.tick_params(axis='x', colors='black', labelsize = 45, width=6, length=15, color='black')
ax.tick_params(axis='y', colors='black', labelsize = 45, width=6, length=15, color='black')

plt.axvline(x=0.1840105, color='tab:red', linewidth = 4, linestyle = '--')
plt.text(0.192, 15.9, 'Prinstine Cu(100): \n0.184 eV', ha='left', va='center', fontsize = 45, color='tab:red')

plt.savefig('{}dim_avg_ads.png'.format(image_directory),dpi=600, bbox_inches = "tight")
plt.show();

# numerical representation of adsorbate averages
difference_data_asc












ModuleNotFoundError: ignored

In [8]:
import nltk
import sklearn

print('The nltk version is {}.'.format(nltk.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))

# The nltk version is 3.0.0.
# The scikit-learn version is 0.15.2.


The nltk version is 3.2.5.
The scikit-learn version is 0.22.2.post1.
